In [4]:
"""
이상치 탐지 문제

이상치 : 통계적으로 다른 데이터들과 큰 편차를 보이는 관측값
"""

import pandas as pd
import numpy as np
from pandas import datetime
from matplotlib import pyplot as plt # 시각화

<ipython-input-4-9dadf96126b8>:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  from pandas import datetime


In [59]:
"""
Load AirQualityUCI Data
"""

# str -> datetime csv -> DateFrame

def parser(x):
    return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

input_file = './data/AirQualityUCI_refined.csv' 

df = pd.read_csv(input_file,
                 index_col=[0], 
                 parse_dates=[0], # 문자열이기 때문에 paser을 해준다
                 date_parser=parser)

df.head()

,CO(GT),PT08.S1(CO),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),RH,AH,C6H6(GT)
Datetime,,,,,,,,,,,
2004-03-10 18:00:00,2.6,1360.0,1046.0,166.0,1056.0,113.0,1692.0,1268.0,48.9,0.7578,11.9
2004-03-10 19:00:00,2.0,1292.0,955.0,103.0,1174.0,92.0,1559.0,972.0,47.7,0.7255,9.4
2004-03-10 20:00:00,2.2,1402.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,54.0,0.7502,9.0
2004-03-10 21:00:00,2.2,1376.0,948.0,172.0,1092.0,122.0,1584.0,1203.0,60.0,0.7867,9.2
2004-03-10 22:00:00,1.6,1272.0,836.0,131.0,1205.0,116.0,1490.0,1110.0,59.6,0.7888,6.5


In [6]:
pip install pyqt5

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Visualization setup
%matplotlib
from matplotlib import pyplot as plt
import seaborn; seaborn.set()  # set plot styles
%config InlineBackend.figure_format = 'svg'
plt.rcParams['figure.figsize'] = [10, 5] 
plt.ion() # enable the interactive mode 

Using matplotlib backend: Qt5Agg


In [12]:
# CO(GT) 이산화탄소 변수를 선택해서 시각화하는 코드

df['CO(GT)'].plot()

<AxesSubplot:xlabel='Datetime'>

In [60]:
# 결측 데이터 처리 : 선형보간
# interpolate 선형보간한 데이터를 리턴

co = df['CO(GT)'].copy()
co.interpolate(inplace=True) #호출한 데이터에 덮어 사용하는 매개변수 inplace

In [62]:
plt.plot(df['CO(GT)'], label = 'original', zorder = 2)
plt.plot(co, label = 'linear interpolation', zorder = 1)
plt.legend(loc = 'best')

In [63]:
"""
IOR = Q3 - Q1
uppper fence(상한) = Q3 + 1.5IQR
lower fence(하한) = Q1 - 1.5IQR

Q3(하위75% = 상위 25%)
Q2(중앙값)
Q1(하위 25%)

maximum : uppper fence 넘지않는 값중에 가장 큰값
minimum : lower fence 보다 작지않는 값중에 작은 값
"""

# Detecting outliers using Boxplot

plt.boxplot(co)
plt.title("Detecting outliers using Boxplot")
plt.xlabel('CO(GT)')

Text(0.5, 0, 'CO(GT)')

In [35]:
# Calculate correlations between variables
# 이상치 시각화 : 산포도 - 상관행렬을 통해 이산화탄소와 상관성이 높은/낮은 변수 선택
# 상관행렬 : 변수쌍 상관성 ex.) 양의 상관성, 음의 상관성 0에 가까울수록 상관성이 적다

corr_matrix = df.corr() # 모든 변수들간에 상관성을 수치화 -1 ~ 1 사이의 값으로 표현
print(corr_matrix)

                 CO(GT)  PT08.S1(CO)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
CO(GT)         1.000000     0.877203       0.914973  0.792557     -0.701703   
PT08.S1(CO)    0.877203     1.000000       0.892964  0.713654     -0.771938   
PT08.S2(NMHC)  0.914973     0.892964       1.000000  0.704435     -0.796703   
NOx(GT)        0.792557     0.713654       0.704435  1.000000     -0.655707   
PT08.S3(NOx)  -0.701703    -0.771938      -0.796703 -0.655707      1.000000   
NO2(GT)        0.679262     0.641529       0.646245  0.763111     -0.652083   
PT08.S4(NO2)   0.639470     0.682881       0.777254  0.233731     -0.538468   
PT08.S5(O3)    0.851403     0.899324       0.880578  0.787046     -0.796569   
RH             0.040218     0.114606      -0.090380  0.221032     -0.056740   
AH             0.065809     0.135324       0.186933 -0.149323     -0.232017   
C6H6(GT)       0.845144     0.883795       0.981950  0.626638     -0.735744   

                NO2(GT)  PT08.S4(NO2)  PT08.S5(O3) 

In [65]:
# Choose the least correlated variable
# 가장 상관성이 적은 RH

rh = df['RH'].copy().interpolate() # RH(Relative Humidity) interpolate 결측치가 메꿔진다
rh.plot()

<AxesSubplot:xlabel='Datetime'>

In [66]:
# Visualize a scatter plot(CO, RH) 특별한 패턴이 존재하지 않는다

plt.scatter(co, rh, s=12, c='black') # s = size c = column
plt.xlabel('CO(GT)') # x축
plt.ylabel('RH') # y축

Text(0, 0.5, 'RH')

In [67]:
# Choose the most correlated variable 
# NMHC 객체 생성
nmhc = df['PT08.S2(NMHC)'].copy().interpolate() # NMHC(non-methane hydrocarbons)

In [68]:
# Visualize a scatter plot(CO, NMHC)
# 상관성이 높은 두변수 CO, NMHC - 시각적으로 이상치를 식별하기 용이함 

plt.scatter(co, nmhc, s=12, c='black')
plt.xlabel('CO(GT)')
plt.ylabel('PT08.S2(NMHC)')

Text(0, 0.5, 'PT08.S2(NMHC)')

In [40]:
"""
IQR-based Outlier Detection
"""

# Q1, Q2(median), Q3 사분위수 범위(IQR) 기반 이상치 탐지
q1 = co.quantile(0.25) # quantile 특정분위값 계산 25& value
median = co.quantile(0.5)  # median
q3 = co.quantile(0.75) # 75% value
print(q1, median, q3)

1.1 1.8 2.9


In [69]:
# IQR, upper_fence, lower_fence (IQR,상한,하한) 계산
iqr = q3-q1
upper_fence = q3 + 1.5*iqr
lower_fence = q1 - 1.5*iqr
print(iqr,upper_fence, lower_fence)

1.7999999999999998 5.6 -1.5999999999999996


In [70]:
# Filtering the outliers
# 이상치 탐지 : 상한 보다 크거나 하한 보다 작은 값 - .loc 인덱서를 이용한 필터링, 음수가 존재할 수 없다
# 모두 upeer_fence 를 넘어간 값 224개

outliers = co.loc[(co > upper_fence) | (co < lower_fence)]
print(outliers)

Datetime
2004-03-11 19:00:00    6.9
2004-03-11 20:00:00    6.1
2004-03-12 20:00:00    6.6
2004-03-14 20:00:00    5.9
2004-03-15 09:00:00    8.1
                      ... 
2005-03-23 19:00:00    6.2
2005-03-23 20:00:00    7.2
2005-03-24 19:00:00    5.9
2005-03-24 20:00:00    7.5
2005-03-25 19:00:00    5.7
Name: CO(GT), Length: 224, dtype: float64


In [71]:
# Mask for outliers
mask = co.index.isin(outliers.index) # isin 메서드
mask[:50]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])

In [72]:
# Visualize the normal data and outliers
plt.plot(co[~mask], label='normal', color='blue', # ~mask : False 값만 가져온다
    marker='o', markersize=3, linestyle='None')
plt.plot(outliers, label='outliers', color='red',
    marker='x', markersize=3, linestyle='None')
plt.legend(loc='best')

In [45]:
# Removing the outliers

co_refined = co.copy()
co_refined[mask] = np.nan
print(co_refined[mask])
co_refined.plot()

Datetime
2004-03-11 19:00:00   NaN
2004-03-11 20:00:00   NaN
2004-03-12 20:00:00   NaN
2004-03-14 20:00:00   NaN
2004-03-15 09:00:00   NaN
                       ..
2005-03-23 19:00:00   NaN
2005-03-23 20:00:00   NaN
2005-03-24 19:00:00   NaN
2005-03-24 20:00:00   NaN
2005-03-25 19:00:00   NaN
Name: CO(GT), Length: 224, dtype: float64


<AxesSubplot:xlabel='Datetime'>

In [46]:
# Linear interpolation for reconstructing outliers removed.
co_refined.interpolate(inplace=True) # 끊어진 데이터를 메꿔준다
co_refined.plot()

<AxesSubplot:xlabel='Datetime'>

In [73]:
"""
Detecting Outliers with Z-Scores
"""

# Visualize the distribution of the 'CO(GT)' variable
import seaborn as sns
sns.distplot(co) # 분포를 보여준다

C:\Users\jaesu\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


<AxesSubplot:xlabel='CO(GT)', ylabel='Density'>

In [52]:
# Mean, Standard deviation
mean = np.mean(co) # 평균
std = np.std(co) # 표준편차
print(mean, std)

2.126146200705357 1.4369815748596482


In [53]:
# Calculate Z-scores for each data points

outliers = [] # 이상치를 저장하기 위한 리스트변수 선언
thres = 3   # Z-score threshold Z-score 값의 절대값이 3을 벗어난 값

for i in co:
    z_score = (i-mean) / std
    if (np.abs(z_score) > thres):
        print(z_score)
        outliers.append(i)

3.3221398818289734
3.113368937755246
4.157223658123882
4.087633343432639
3.0437786230640036
3.113368937755246
3.8092720846676698
3.182959252446488
3.113368937755246
3.600501140593942
3.5309108259027
3.0437786230640036
3.0437786230640036
3.182959252446488
3.0437786230640036
3.600501140593942
3.182959252446488
3.7396817699764275
3.0437786230640036
3.182959252446488
3.3917301965202156
4.087633343432639
3.878862399358912
3.2525495671377302
4.36599460219761
3.2525495671377302
4.018043028741397
4.36599460219761
4.087633343432639
4.36599460219761
5.131488063801275
3.0437786230640036
3.113368937755246
3.7396817699764275
3.2525495671377302
4.018043028741397
4.087633343432639
4.922717119727547
3.3917301965202156
3.786075313103922
4.180420429687629
4.574765546271336
4.157223658123882
4.087633343432639
3.113368937755246
3.2525495671377302
3.9484527140501537
5.618620266639971
3.113368937755246
3.7396817699764275
3.5309108259027
4.296404287506367
4.505175231580093
3.3221398818289734
3.04377862306400

In [54]:
# Simplified version of filtering outliers
outliers = co.loc[np.abs((co-mean)/std) > 3].copy() # 116 개
outliers

Datetime
2004-03-11 19:00:00    6.9
2004-03-12 20:00:00    6.6
2004-03-15 09:00:00    8.1
2004-03-15 19:00:00    8.0
2004-03-15 20:00:00    6.5
                      ... 
2005-03-14 19:00:00    6.5
2005-03-14 20:00:00    6.5
2005-03-15 20:00:00    7.4
2005-03-23 20:00:00    7.2
2005-03-24 20:00:00    7.5
Name: CO(GT), Length: 116, dtype: float64

In [55]:
# Mask for outliers
mask = co.index.isin(outliers.index)
mask

array([False, False, False, ..., False, False, False])

In [74]:
# Comparison of distributions before/after outlier removal
sns.distplot(co, axlabel='CO(GT)', label='original')
sns.distplot(co[~mask], label='outliers removed')
plt.legend(loc='best')

C:\Users\jaesu\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
C:\Users\jaesu\anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [75]:
# [exer] Adjust thres

# Flooring and Capping
floor = co.quantile(0.1) # Flooring : 작은 값을 가지는 이상치들을 낮은 분위수의 값으로 대체
cap = co.quantile(0.9) # Capping : 큰 값을 가지는 이상치들을 높은 분위수의 값으로 대체

co.loc[co < floor] = floor
co.loc[co > cap] = cap

In [76]:
# Visualize the result
co.plot()

<AxesSubplot:xlabel='Datetime'>